In [ ]:
from google.colab import files
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU, Flatten, Reshape, Dense
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import os
from PIL import Image
import gc

def upload_image():
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f'User uploaded file "{filename}"')
        return filename
    return None

def load_image(image_path, target_size):
    try:
        print(f"Loading image from path: {image_path}")
        img = Image.open(image_path).convert('RGB')
        img = img.resize(target_size)
        img = np.array(img) / 127.5 - 1.0
        return img
    except IOError as e:
        print(f"Error processing image: {e}")
        return None

print("Upload content image:")
content_image_path = upload_image()
print("Upload style image:")
style_image_path = upload_image()

print(f"Content image path: {content_image_path}")
print(f"Style image path: {style_image_path}")


content_image = load_image(content_image_path, (64, 64))
style_image = load_image(style_image_path, (64, 64))


if content_image is None:
    raise RuntimeError(f"Failed to load or process the content image from path: {content_image_path}")

if style_image is None:
    raise RuntimeError(f"Failed to load or process the style image from path: {style_image_path}")


content_image = np.expand_dims(content_image, axis=0)
style_image = np.expand_dims(style_image, axis=0)

def build_generator():
    inputs = Input(shape=(64, 64, 3))
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(16, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(3, (3, 3), padding='same', activation='tanh')(x)
    model = Model(inputs, x)
    return model

def build_discriminator():
    inputs = Input(shape=(64, 64, 3))
    x = Conv2D(16, (3, 3), padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs, x)
    return model

def build_cgan(generator, discriminator):
    inputs = Input(shape=(64, 64, 3))
    gen_output = generator(inputs)
    disc_output = discriminator(gen_output)
    model = Model(inputs, disc_output)
    return model

def compile_models(generator, discriminator, cgan):
    discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    discriminator.trainable = False
    cgan.compile(optimizer='adam', loss='binary_crossentropy')

def train(generator, discriminator, cgan, epochs=1000, batch_size=1):
    for epoch in range(epochs):

        noise = np.random.normal(0, 1, (batch_size, 64, 64, 3))
        fake_images = generator.predict(noise)


        real_images = np.tile(content_image, (batch_size, 1, 1, 1))
        combined_images = np.concatenate([real_images, fake_images], axis=0)
        labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))], axis=0)


        d_loss = discriminator.train_on_batch(combined_images, labels)


        g_loss = cgan.train_on_batch(noise, np.ones((batch_size, 1)))

        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]}] [G loss: {g_loss}]")
            save_images(generator, epoch)

def save_images(generator, epoch, image_path='images/'):
    if not os.path.exists(image_path):
        os.makedirs(image_path)
    noise = np.random.normal(0, 1, (1, 64, 64, 3))
    generated_image = generator.predict(noise)[0]
    plt.imshow((generated_image + 1) / 2)
    plt.title(f'Epoch {epoch}')
    plt.savefig(f"{image_path}epoch_{epoch}.png")
    plt.close()


generator = build_generator()
discriminator = build_discriminator()
cgan = build_cgan(generator, discriminator)


compile_models(generator, discriminator, cgan)


train(generator, discriminator, cgan, epochs=1000, batch_size=1)


del generator, discriminator, cgan
gc.collect

Upload content image:


Saving b3.jpg to b3.jpg
User uploaded file "b3.jpg"
Upload style image:


Saving apple.jpg to apple.jpg
User uploaded file "apple.jpg"
Content image path: b3.jpg
Style image path: apple.jpg
Loading image from path: b3.jpg
Loading image from path: apple.jpg


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0 [D loss: 0.6274022459983826 | D accuracy: 0.5] [G loss: [array(0.62740225, dtype=float32), array(0.62740225, dtype=float32), array(0.5, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━

<function gc.collect(generation=2)>